In [1]:
from imdb import IMDb
import pandas as pd
from PIL import Image
from StringIO import StringIO
import requests
import os
import time
from shutil import copyfile
import csv

In [2]:
dir_python_notebook = os.getcwd()
dir_movie_project = os.path.abspath(os.path.join(dir_python_notebook, os.pardir))
dir_data = os.path.join(dir_movie_project, 'data')

In [8]:
imdb_filename = str(dir_data)+'\\imdb_imputed.csv'
imdb_movies = pd.read_csv(imdb_filename,index_col=0, sep=',', encoding='utf-8', quoting=csv.QUOTE_NONNUMERIC)

In [9]:
imdb_movies.head(5)

,art direction,assistant director,cast,casting director,certificates,cinematographer,color info,costume department,costume designer,countries,...,production manager_1,set decoration_1,sound crew_1,stunt performer_1,visual effects_1,writer_1,cast_1,cast_2,cast_3,cast_4
100.0,0,"[u'0123407', u'0717230', u'0006859']","[u'0002076', u'0002077', u'0602941', u'0005458...","[u'0288911', u'0005363']","[u'Argentina:13', u'Australia:MA15+', u'Brazil...",[u'0362165'],[u'Color'],"[u'0989056', u'0788235']",[u'0171871'],[u'UK'],...,92061.0,405176.0,30552.0,337040.0,91100.0,5363.0,2076.0,2077.0,602941.0,5458.0
10001.0,"[u'0264966', u'0316904', u'0383595', u'0550026']","[u'0267087', u'0382238', u'0680614', u'0731968...","[u'0000635', u'0494189', u'0397398', u'0933957...",0,"[u'Australia:PG', u'Finland:K-8', u'Germany:12...",[u'0201372'],[u'Color'],"[u'0100748', u'1021483', u'0372320']","[u'0100748', u'0915204']",[u'Australia'],...,55373.0,0.0,66740.0,49882.0,184789.0,730000.0,635.0,494189.0,397398.0,933957.0
10002.0,[u'0413541'],"[u'0013936', u'0110556', u'0179192']","[u'0001364', u'0879154', u'0000323', u'0001059...",[u'0007109'],"[u'Argentina:18', u'Australia:M', u'Finland:K-...",[u'0695536'],[u'Color::(Technicolor)'],"[u'0196382', u'0788827']",[u'0296220'],[u'UK'],...,115536.0,0.0,60952.0,286157.0,0.0,1403.0,1364.0,879154.0,323.0,1059.0
10003.0,"[u'0144261', u'0658390', u'0724682', u'0866777']","[u'0068925', u'0072593', u'0250856', u'0250856...","[u'0000174', u'0000223', u'0784884', u'0631792...","[u'0505059', u'0689691']","[u'Argentina:13', u'Australia:M', u'Brazil:12'...",[u'0005793'],[u'Color::(Rankcolor)'],"[u'0183262', u'0232875', u'0412130', u'0456586...",[u'0829641'],[u'USA'],...,97161.0,949952.0,10057.0,132625.0,463671.0,153546.0,174.0,223.0,784884.0,631792.0
10004.0,[u'0003488'],"[u'0551862', u'0581688', u'0815340']","[u'0000643', u'0001836', u'0001272', u'0001164...",[u'0470948'],"[u'Argentina:16', u'Australia:MA15+', u'Denmar...",[u'0003791'],[u'Color'],"[u'0124203', u'0305128', u'0568764']",[u'0624703'],[u'USA'],...,665655.0,130700.0,123638.0,7200.0,84624.0,175.0,643.0,1836.0,1272.0,1164.0


In [22]:
imdb_genres = ['Action','Adult','Adventure','Animation','Biography','Comedy','Crime','Documentary','Drama','Family',
              'Fantasy','Film Noir','Game-Show','History','Horror','Musical','Music','Mystery','News','Reality-TV',
              'Romance','Sci-Fi','Short','Sport','Talk-Show','Thriller','War','Western']


In [24]:
len(imdb_genres)

28

In [13]:
def get_genre(tmdb_movies ,key):
    tmdb_genre = tmdb_movies[tmdb_movies[key].notnull()][key].tolist()
    tmdb_genre_set = set()
    for g in tmdb_genre:
        if g is not None:
            tmdb_genre_set = tmdb_genre_set.union(set(g))
    tmdb_genre = list(tmdb_genre_set)
    tmdb_genre.sort()
    return(tmdb_genre)

In [14]:
def get_genere_num (row, column_name):
    if row[column_name] is None :
        return 0
    else:
        return len(row[column_name])

In [15]:
def is_genre (row, column_name, genre):
    """check if that movie is in this genre as a movie can have more than 1 genre"""
    if row[column_name] is None :
        return 0
    else:
        if genre in row[column_name] :
            return 1
        else:
            return 0

In [16]:
def get_all_genre (row, column_name, imdb_genre):
    """check a vector for all movie genres"""
    s = ""
    for g in imdb_genre:
        if row[column_name] is None :
            s = s +"0"
        else:
            if g in row[column_name] :
                s = s +"1"
            else:
                s = s +"0"
    return s

In [23]:
imdb_genre = imdb_genres
imdb_genre.sort()
imdb_movies[u'genre_num'] = imdb_movies.apply(lambda row: get_genere_num(row,u'genres'), axis=1)

for g in imdb_genre: 
    imdb_movies[g] = imdb_movies.apply(lambda row: is_genre(row,u'genres',g), axis=1)

In [25]:
imdb_movies['all_genre'] = imdb_movies.apply(lambda row: get_all_genre(row,u'genres',imdb_genre), axis=1)

In [26]:
imdb_movies['all_genre'] = imdb_movies['all_genre'].astype('str')

In [32]:
imdb_filename = str(dir_data)+'\\imdb_imputed_processed.csv'
imdb_movies.to_csv(imdb_filename, sep=',', encoding='utf-8')

In [ ]:
imdb_filename = str(dir_data)+'\\imdb_imputed_processed.json'
imdb_movies.to_json(path_or_buf= imdb_filename)

In [30]:
imdb_movies.columns

Index([            u'art direction',        u'assistant director',
                            u'cast',          u'casting director',
                    u'certificates',           u'cinematographer',
                      u'color info',        u'costume department',
                u'costume designer',                 u'countries',
                        u'director',              u'distributors',
                          u'editor',                    u'genres',
                         u'imdb_id',                 u'languages',
                         u'make up',   u'miscellaneous companies',
              u'miscellaneous crew',            u'original music',
                            u'plot',                  u'producer',
            u'production companies',        u'production manager',
                          u'rating',                  u'runtimes',
                  u'set decoration',                u'sound crew',
                 u'stunt performer',                   u'tmdb_